In [1]:
from math import factorial
from itertools import combinations

import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import t
from sklearn.svm import SVC
import plotly.express as px 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.datasets import make_moons
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold

import utils

# The Data

In [2]:
X, y = make_moons(noise=0.352, random_state=1, n_samples=600)

In [4]:
utils.plot_dataset(X,y)

# Running GridSearch

In [5]:
search, n, n_train, n_test = utils.fit_search(X, y, folds=10, repetitions=10)

In [6]:
results_df = utils.get_results_df(search)

## Showing Correlation of Folds

In [8]:
# create df of model scores ordered by performance
model_scores = results_df.filter(regex=r"split\d*_test_score")


In [9]:
utils.plot_cv_test(model_scores)

In [10]:
utils.plot_correlation_heatmap(model_scores)

# Comparing two models: frequentist approach

In [12]:
utils.frequentist_two_model(model_scores, n, n_train, n_test)

,Type,Model Name,t-value,p-value
0,Corrected,linear,3.27,0.001
1,Uncorrected,2_poly,11.38,0.000


# Comparing two models: Bayesian approach

In [13]:
utils.plot_bayesian_posterior(model_scores, n, n_train, n_test)

IndexError: cannot do a non-empty take from an empty axes.

## Region of Practical Equivalence

In [278]:
def plot_rope(model_scores: pd.DataFrame, n: int, n_train: int, n_test: int, rope_interval: float):
    model_1_scores = model_scores.iloc[0].values  # scores of the best model
    model_2_scores = model_scores.iloc[1].values  # scores of the second-best model
    differences = model_1_scores - model_2_scores
    df = n - 1

    # initialize random variable
    t_post = t(
        df, loc=np.mean(differences), scale=utils.corrected_std(differences, n_train, n_test)
    )
    x = np.linspace(t_post.ppf(0.001), t_post.ppf(0.999), 10001)
    t_post_values = t_post.pdf(x)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=x,
            y=t_post_values,
            mode='lines',
            name='Posterior Distribution',
            showlegend=False
        )
    )
    x_int = [xc for xc in x if xc >= rope_interval[0] and xc <= rope_interval[1]]
    y_int = t_post.pdf(x_int)

    x_annot = np.quantile(x_int, 0.5)
    y_annot = t_post.pdf(x_annot) / 2
    rope_prob = t_post.cdf(rope_interval[1]) - t_post.cdf(rope_interval[0])

    fig.add_trace(
        go.Scatter(
            x=x_int,
            y=y_int,
            mode='none',
            fill='tozeroy',
            fillcolor="rgba(99, 110, 250, 0.5)",
            
            showlegend=False
        )
    )

    fig.add_vline(
        x=rope_interval[0],
        line_dash='dot'
    )

    fig.add_vline(
        x=rope_interval[1],
        line_dash='dot'
    )

    fig.add_annotation(
        x=x_annot,
        y=y_annot,
        text=f"<b>{100*rope_prob:.2f}%</b>",
        showarrow=False,
        font=dict(
            size=18
        )
    )

    fig.update_layout(
        xaxis_title='Mean Difference (μ)',
        yaxis_title='Probability Density',
        title=f'Region of Practical Equivalence (ROPE)',
        yaxis_showticklabels=False
    )

    return fig


In [279]:
plot_rope(model_scores, n, n_train, n_test, rope_interval=(-0.01, 0.01))

In [280]:
def get_cred_intervals(intervals: list[float], model_scores: pd.DataFrame, n: int, n_train: int, n_test: int):
    model_1_scores = model_scores.iloc[0].values  # scores of the best model
    model_2_scores = model_scores.iloc[1].values  # scores of the second-best model
    differences = model_1_scores - model_2_scores
    df = n - 1

    # initialize random variable
    t_post = t(
        df, loc=np.mean(differences), scale=utils.corrected_std(differences, n_train, n_test)
    )
    cred_intervals = []

    for interval in intervals:
        cred_interval = list(t_post.interval(interval))
        cred_intervals.append([interval, cred_interval[0], cred_interval[1]])

    cred_int_df = pd.DataFrame(
        cred_intervals, columns=["interval", "lower value", "upper value"]
    ).set_index("interval")
    
    return cred_int_df

In [281]:
get_cred_intervals([0.5, 0.75, 0.95], model_scores, n, n_train, n_test)

,lower value,upper value
interval,,
0.50,0.000977,0.019023
0.75,-0.005422,0.025422
0.95,-0.016445,0.036445


# Pairwise comparison of all models: frequentist approach

In [282]:
def get_model_scores_pairs(model_scores: pd.DataFrame, idx: tuple[int, int]) -> list[tuple[np.array, str]]:
        idx1, idx2 = idx
        model_1_scores = model_scores.iloc[idx1].values  # scores of the best model
        model_2_scores = model_scores.iloc[idx2].values  # scores of the second-best model
        # Getting Name of the models
        model_1_name = model_scores.index[idx1]
        model_2_name = model_scores.index[idx2]

        return (model_1_scores, model_1_name), (model_2_scores, model_2_name)

In [285]:
def get_pairwise_frequentist(model_scores: pd.DataFrame, n: int, n_train: int, n_test: int) -> pd.DataFrame:
    n_comparisons = factorial(len(model_scores)) / (
        factorial(2) * factorial(len(model_scores) - 2)
    )
    df = n - 1

    cmbs = list(combinations(range(len(model_scores)), 2))
    pairwise_t_test = []

    for cmb in cmbs:
        (model_i_scores, model_i_name), (model_k_scores, model_k_name) = get_model_scores_pairs(model_scores, cmb)
        differences = model_i_scores - model_k_scores
        
        assert n == differences.shape[0], "Number of samples is not equal to the number of differences"

        t_stat, p_val = utils.compute_corrected_ttest(differences, df, n_train, n_test)
        p_val *= n_comparisons  # implement Bonferroni correction
        # Bonferroni can output p-values higher than 1
        p_val = 1 if p_val > 1 else p_val
        pairwise_t_test.append(
            [model_i_name, model_k_name, t_stat, p_val]
        )
    pairwise_comp_df = pd.DataFrame(
        pairwise_t_test, columns=["model_1", "model_2", "t_stat", "p_val"]
    ).round(3)

    return pairwise_comp_df

In [286]:
get_pairwise_frequentist(model_scores, n, n_train, n_test)

,model_1,model_2,t_stat,p_val
0,rbf,linear,0.750,1.000
1,rbf,3_poly,1.657,0.302
2,rbf,2_poly,4.565,0.000
3,linear,3_poly,1.111,0.807
4,linear,2_poly,4.276,0.000
5,3_poly,2_poly,3.851,0.001


# Pairwise comparison of all models: Bayesian approach

In [291]:
def get_pairwise_bayesian(rope_interval: tuple[float, float], model_scores: pd.DataFrame, n: int, n_train: int, n_test: int) -> pd.DataFrame:
    df = n - 1

    cmbs = list(combinations(range(len(model_scores)), 2))
    pairwise_bayesian = []

    for cmb in cmbs:
        (model_i_scores, model_i_name), (model_k_scores, model_k_name) = get_model_scores_pairs(model_scores, cmb)
        differences = model_i_scores - model_k_scores

        assert n == differences.shape[0], "Number of samples is not equal to the number of differences"

        t_post = t(
            df, loc=np.mean(differences), scale=utils.corrected_std(differences, n_train, n_test)
        )

        worse_prob = t_post.cdf(rope_interval[0])
        better_prob = 1 - t_post.cdf(rope_interval[1])
        rope_prob = t_post.cdf(rope_interval[1]) - t_post.cdf(rope_interval[0])
        t_stat, p_val = utils.compute_corrected_ttest(differences, df, n_train, n_test)
        
        pairwise_bayesian.append([model_i_name, model_k_name, t_stat, p_val, worse_prob, better_prob, rope_prob])

    pairwise_bayesian_df = pd.DataFrame(
        pairwise_bayesian, columns=["model_1", "model_2", "t_stat", "p_val", "worse_prob", "better_prob", "rope_prob"]
    ).round(3)

    return pairwise_bayesian_df

In [292]:
get_pairwise_bayesian((-0.01, 0.01), model_scores, n, n_train, n_test)

,model_1,model_2,t_stat,p_val,worse_prob,better_prob,rope_prob
0,rbf,linear,0.750,0.227,0.068,0.500,0.432
1,rbf,3_poly,1.657,0.050,0.018,0.882,0.100
2,rbf,2_poly,4.565,0.000,0.000,1.000,0.000
3,linear,3_poly,1.111,0.135,0.063,0.750,0.187
4,linear,2_poly,4.276,0.000,0.000,1.000,0.000
5,3_poly,2_poly,3.851,0.000,0.000,1.000,0.000
